# Pohualli Interactive Notebook (Colab Compatible)
This notebook launches the FastAPI web UI inline via an iframe while also providing Python helper functions to query the API directly.

**Features**:
- Start the Pohualli FastAPI server (uvicorn) in a background thread.
- Display the existing HTML UI (same as running locally) inside the notebook.
- Provide helper functions to perform conversions and auto-correction derivations programmatically.

> If running on **Google Colab**, the first cell will install dependencies. The UI will appear below after the server starts.

In [ ]:
# Run backend and show frontend UI in one cell
import sys, os, threading, time

def in_colab():
    try:
        import google.colab  # type: ignore
        return True
    except Exception:
        return False

REQ = 'fastapi uvicorn jinja2 httpx pyngrok'
if in_colab():
    print('Colab detected: installing packages...')
    import subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', REQ])

REPO_URL = 'https://github.com/muscariello/pohualli-python.git'
if in_colab() and not os.path.isdir('pohualli-python'):
    os.system(f'git clone --depth 1 {REPO_URL}')
    os.chdir('pohualli-python')
if 'pohualli' not in sys.modules:
    sys.path.append('.')
from pohualli.webapp import app

import uvicorn
SERVER_STARTED = False

def run_server():
    uvicorn.run(app, host='0.0.0.0', port=8000, log_level='warning')

if not SERVER_STARTED:
    th = threading.Thread(target=run_server, daemon=True)
    th.start()
    SERVER_STARTED = True
    time.sleep(2)

from IPython.display import IFrame, display
if in_colab():
    from pyngrok import ngrok
    public_url = ngrok.connect(8000)
    print('Web UI:', public_url)
    display(IFrame(src=public_url, width="100%", height=900))
else:
    display(IFrame(src='http://127.0.0.1:8000/', width='100%', height=900))
print('Pohualli Web UI is running above!')

In [ ]:
# If running in Google Colab, install dependencies (FastAPI, Uvicorn, etc.)
import os, sys, subprocess, textwrap, time
REQ = 'fastapi uvicorn jinja2 httpx'
def in_colab():
    try:
        import google.colab  # type: ignore
        return True
    except Exception:
        return False
if in_colab():
    print('Running in Colab: installing packages...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', REQ])
else:
    print('Not in Colab (assuming local dev environment has deps).')

In [ ]:
# Import project (assumes repository cloned / present).
# If in Colab and repo not present, clone the GitHub repository.
REPO_URL = 'https://github.com/muscariello/pohualli-python.git'
if in_colab() and not os.path.isdir('pohualli-python'):
    !git clone --depth 1 $REPO_URL
    os.chdir('pohualli-python')
# Ensure project path is on sys.path
if 'pohualli' not in sys.modules:
    sys.path.append('.')
from pohualli.webapp import app  # FastAPI application
print('Loaded FastAPI app:', app.title)

In [ ]:
# Launch the FastAPI app with uvicorn in a background thread.
import threading
import uvicorn
SERVER_STARTED = False
def run_server():
    uvicorn.run(app, host='0.0.0.0', port=8000, log_level='warning')
if not SERVER_STARTED:
    th = threading.Thread(target=run_server, daemon=True)
    th.start()
    SERVER_STARTED = True
    time.sleep(2)  # Allow server to start
print('Server should now be running on port 8000.')

### Embedded UI
If you are in Colab, the iframe below will *not* directly render a localhost server. Colab notebooks run remotely; for Colab you would need to use `pyngrok` or Cloud Run to expose the service.

Locally (Jupyter), this iframe will show the full Pohualli HTML interface.

In [ ]:
from IPython.display import IFrame, display, HTML
if in_colab():
    display(HTML('<p><b>Colab detected:</b> Localhost iframe is not accessible. Use tunnel instructions below.</p>'))
else:
    display(IFrame(src='http://127.0.0.1:8000/', width='100%', height=900))

### Optional: Expose Server Publicly in Colab
To surface the FastAPI UI in Colab, you can open a tunnel using `pyngrok`:
```python
!pip install pyngrok
from pyngrok import ngrok
public_url = ngrok.connect(8000)
public_url
```
Then open the printed HTTPS URL to access the UI.

### Helper Functions
Use these to query the API endpoints programmatically.

In [ ]:
import httpx, json
BASE = 'http://127.0.0.1:8000'
def convert(jdn: int, **overrides):
    params = {'jdn': jdn, **overrides}
    r = httpx.get(BASE + '/api/convert', params=params, timeout=10)
    r.raise_for_status()
    return r.json()
def derive(jdn: int, **spec):
    params = {'jdn': jdn, **spec}
    r = httpx.get(BASE + '/api/derive-autocorr', params=params, timeout=30)
    r.raise_for_status()
    return r.json()
print('Helper functions ready: convert(jdn), derive(jdn, ...)')

In [ ]:
# Example usage
sample = convert(2451545)  # J2000 epoch as demonstration
sample['long_count'], sample['tzolkin_value'], sample['tzolkin_name']

In [ ]:
# Derive example (fill only what you know)
derive_example = derive(2451545, tzolkin='11 Ik')
derive_example

### Shutdown (Optional)
The server thread is daemonized and will terminate when the kernel stops.